# Eliminación de preguntas repetidas
Ya con todas las preguntas que cumplen los requisitos del INE para ser tomadas en cuenta, vamos 
a hacer otra depuración para eliminar aquellas preguntas que están repetidas.
Es importante considerar que dos preguntas pueden ser iguales aún cuando estén formuladas con 
palabras diferentes (es decir, una puede ser paráfrasis de la otra).
El resultado de este paso serán dos archivos:

    • Uno que contenga el texto de las preguntas repetidas, así como una métrica que mida su 
      parecido. De estas preguntas, sólo se conservará 1 en el dataset de trabajo.

        ◦ La pregunta que será conservada es aquella que fue enviada primero de acuerdo al 
          timestamp
          
    • Un archivo con las preguntas depuradas (eliminando las repeticiones)

In [ ]:
import pandas as pd